<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Network graph

## Input

### Import libraries

In [ ]:
import glob
import pandas as pd
from pyvis.network import Network

## Model

### Get all notebooks

In [ ]:
def get_all_notebooks():
    # Init
    notebooks = []
    
    # Regex to get all files
    regex = f"{ROOT_PATH}/**/*"
    
    # Filter on notebooks
    for file in glob.glob(regex, recursive=True):
        if file.endswith(".ipynb"):
            notebooks.append(file)
    return notebooks

### Create inputs for pyvis

In [ ]:
def format_df_pyvis(notebooks):
    # Get list of notebooks
    df = pd.DataFrame(notebooks).rename(columns={0: "FILE_PATH"})
    
    # Create columns
    df["ID"] = df.index.astype(str)
    df["NAME"] = df.apply(lambda row: row["FILE_PATH"].split("/")[-1].replace(".ipynb", "").replace("_", " ").strip(), axis=1)
    df["FOLDER"] = df.apply(lambda row: row["FILE_PATH"].split("/")[-2], axis=1)
    
    # Apply color to folder
    df.loc[:, "COLOR"] = "black"
    df.loc[df["FOLDER"] == "models", "COLOR"] = "blue"
    df.loc[df["FOLDER"] == "utils", "COLOR"] = "green"
   
    # Init ID to create nodes
    settings = df.loc[df["NAME"] == "settings", "ID"].values[0]
    pipeline = df.loc[df["NAME"] == "pipeline", "ID"].values[0]
    utils = df.loc[df["NAME"] == "utils", "ID"].values[0]
    
    # Create edges lists
    edges = []
    for index, row in df.iterrows():
        uid =  int(row['ID'])
        if row["NAME"] == "pipeline":
            edge = (uid, int(settings))
            edges.append(edge)
        elif row["NAME"] == "utils":
            edge = (uid,  int(settings))
            edges.append(edge)
        elif row["NAME"] != "pipeline" and row["FOLDER"] == "models":
            edge = (uid,  int(pipeline))
            edges.append(edge)
            edge = (uid,  int(utils))
            edges.append(edge)
        elif row["NAME"] != "utils" and row["FOLDER"] == "utils":
            edge = (uid, int(utils))
            edges.append(edge)
    return df, edges

### Create network with pyvis

In [ ]:
def create_network(
    df,
    edges,
    file_name="project_network.html"
):
    # Init
    net = Network(
        notebook=True,
        height='750px',
        width='100%',
        bgcolor='#222222',
        font_color='white'
    )
    
    # Add add_nodes
    net.add_nodes(
        df["ID"].astype(int).tolist(),
        label=df["NAME"].tolist(),
        color=df["COLOR"].tolist(),
    )
    
    # Add edges
    net.add_edges(edges)
    
    # Save and display network
    network = net.show(file_name)
    return network

### Generate network

In [ ]:
def generate_network():
    # step 1:
    notebooks = get_all_notebooks()
    
    # step 2:
    df, edges = format_df_pyvis(notebooks)
    
    # step 3:
    network = create_network(df, edges)
    return network

## Output

### Display output

In [ ]:
print(f"✅ Network graph helper loaded successfully.")